### Hospital Nurses Scheduling

A hospital service director is responsible for organizing the nurses’ schedules.
A working day is divided into 12 time slots of two hours each. 
Staffing needs vary from one time slot to another. 
For example, few nurses are needed during the night, but the staff must be reinforced in the morning to ensure patient care. 
The table beside gives the staffing needs for each of the time slots.
What is the minimum number of nurses needed to cover time slot requirements? It’s important to know that each nurse works 8 hours a day and has a 2-hours break after 4 hours of work.

<img src = "image03.png" alt = "Nurse time schedule" width = "350">

### Decision variables
$X_i = \text{Minimum nurses needed per shift } i, \quad \text{for } i = 1, 2, \dots, 12$

### Objective Function
Minimize the number of nurses per shift with satisfying all constrains \
$\text{Minimize} \quad Z = \sum_{i=1}^{12} X_i$

### Constraints
The sum of each (shift(i), the shift 2 hours before, 6 hours before and 8 hours before) must be greater or equal to the shift (i) min requirement nurses
\begin{align}
& X_i + X_{i-1} + X_{i-3} + X_{i-4} \ge \text{MinReq}_i \\
& \text{for each shift } i \in \{1, 2, \dots, 12\}, \\
& \text{indices taken modulo 12 (e.g.\ } X_{0} \equiv X_{12}).
\end{align}

In [1]:
#import all the needed library
import gurobipy as gp
from gurobipy import GRB

In [2]:
#prepare data

shifts_data = {"6am-8am": 35,
             "8am-10am": 40,
             "10am-12pm": 40,
             "12pm-2pm": 35,
             "2pm-4pm": 30,
             "4pm-6pm": 30,
             "6pm-8pm": 35,
             "8pm-10pm": 30,
             "10pm-12am": 20,
             "12am-2am": 15,
             "2am-4am": 15,
             "4am-6am": 15}
min_req = list(shifts_data.values())
shift_names = list(shifts_data.keys())
N = len(shift_names)

In [3]:
#build model
model = gp.Model("shift_scheduling")


#create decision variables
X = model.addVars(shift_names, vtype=GRB.INTEGER, name="num_nurses_per_shift")

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2713610
Academic license 2713610 - for non-commercial use only - registered to 44___@std.psau.edu.sa


In [4]:
#objective function
model.setObjective(sum(X[shift] for shift in shift_names), GRB.MINIMIZE)

In [5]:
#add constraints
for i in range(N):
    shifts_to_consider = [
        shift_names[i],
        shift_names[(i - 1) % N],
        shift_names[(i - 3) % N],
        shift_names[(i - 4) % N]
    ]
    model.addConstr(sum(X[shift] for shift in shifts_to_consider) >= min_req[i]
                    )


In [6]:
#turn off the solver output
gp.setParam('OutputFlag', 0)

#optimize the model
model.optimize()

#print the results
for shift in shift_names:
    print(f"{shift}: {X[shift].X:.0f} nurses")
print(f"Optimal total number of nurses: {model.ObjVal:.0f}")

6am-8am: 20 nurses
8am-10am: 20 nurses
10am-12pm: 5 nurses
12pm-2pm: 5 nurses
2pm-4pm: 10 nurses
4pm-6pm: 10 nurses
6pm-8pm: 15 nurses
8pm-10pm: -0 nurses
10pm-12am: -0 nurses
12am-2am: -0 nurses
2am-4am: -0 nurses
4am-6am: 15 nurses
Optimal total number of nurses: 100
